## Домашнее задание

1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [259]:
import numpy as np

In [260]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [261]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [262]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [394]:
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

In [298]:
def calc_logloss(y, y_pred):
    # Изменение
    # Позаимствуем метод обхода логарифма от нуля у библиотеки scikitlearn, а именно clipping =)
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1 - eps)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [265]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.164252033486018

In [266]:
# Плохой пример применения (теперь тоже работает)

y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.2])
calc_logloss(y1, y_pred1)

0.11157177565710535

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.


При использовании модифицированной функции calc_logloss можно брать сколь угодно большие скорости обучения и количества итераций, но log loss будет стремиться к 0, постоянно убывая (проверено на значениях iterations = 10000 и eta = 40. Подбор буду осуществлять на оригинальном коде.

In [436]:
def calc_logloss_2(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [437]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [438]:
def eval_model_2(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]

    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss_2(y, y_pred)

        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % 100 == 0:
            print(f'Iteration = {i}, Weight = {W}, Error = {err}')
    return W

Сначала подбирался параметр скорости обучения, затем количество итераций. Минимального значения, до сваливания в ошибку получилось достичь после 5500 итераций при скорости обучения = 5.

In [439]:
iterations = 5501
eta_test = 5

In [440]:
W = eval_model_2(X_st, y, iterations, eta_test)

Iteration = 0, Weight = [-1.40022026 -7.39264425  0.51078644 -1.30770149], Error = 1.1785958344356262
Iteration = 100, Weight = [-14.42116087  -6.28104638   1.50430271  20.14877699], Error = 0.8242173408675708
Iteration = 200, Weight = [-23.67230529  -7.31994095  -3.89264328  22.78628394], Error = 0.7983723120964876
Iteration = 300, Weight = [-27.55331732  -3.05193129  -7.09618596  23.12211961], Error = 0.13337672340681184
Iteration = 400, Weight = [-29.723293    -3.09355345  -7.90612742  24.58322249], Error = 0.12535956242626906
Iteration = 500, Weight = [-31.32069311  -3.22958293  -8.36802298  25.85947687], Error = 0.12065914563762077
Iteration = 600, Weight = [-32.67264268  -3.45206575  -8.63242156  27.14759072], Error = 0.11717278975339813
Iteration = 700, Weight = [-33.50399014  -3.40627271  -9.00741596  27.58454765], Error = 0.11486092970998182
Iteration = 800, Weight = [-34.21712122  -3.46945859  -9.20939879  28.1582381 ], Error = 0.11309487255472887
Iteration = 900, Weight = [-

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).


In [441]:
def calc_pred_proba(W, X):
    y_pred_proba = np.squeeze(sigmoid(np.dot(X, W)))
    return y_pred_proba

In [445]:
pred = calc_pred_proba(W, X_st)
pred

(10,)

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [446]:
def calc_pred(W, X):

    y_predicted = np.zeros(np.shape(X)[0])

    y_pred_proba = np.squeeze(sigmoid(np.dot(X, W)))

    for i in range(len(y_pred_proba)):
        if y_pred_proba[i] > 0.5:
            y_predicted[i] = 1
        elif y_pred_proba[i] <= 0.5:
            y_predicted[i] = 0

    return y_predicted

In [447]:
calc_pred(W, X_st)

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [470]:
y = np.array([1, 1, 0, 1, 0, 0, 1, 0, 1, 0])
y_pred = np.array([1, 1, 1, 0, 1, 0, 1, 0, 1, 0])

In [471]:
def accuracy(y_true, y_pred):
    score = 0
    for i in range(y_true.shape[0]):
        if y_true[i] == y_pred[i]:
            score += 1
    return score/y_true.shape[0]

In [472]:
accuracy(y, y_pred)

0.7

In [473]:
def conf_matrix(y_true, y_pred):
    tp, fp, fn, tn = 0,0,0,0
    for i in range(y_true.shape[0]):
        if y_true[i] == 1 and  y_pred[i] == 1:
            tp+=1
        elif y_true[i] == 0 and  y_pred[i] == 1:
            fp+=1
        elif y_true[i] == 1 and  y_pred[i] == 0:
            fn+=1
        else:
            tn+=1
    matrix = np.array([[tp, fp], [fn, tn]])
    return matrix


In [474]:
conf_matrix(y, y_pred)


array([[4, 2],
       [1, 3]])

In [476]:
def precision(y_true, y_pred):
    matrix = conf_matrix(y_true, y_pred)

    return matrix[0][0]/(matrix[0][0] + matrix[0][1])

In [477]:
precision(y, y_pred)

0.6666666666666666

In [478]:
def recall(y_true, y_pred):
    matrix = conf_matrix(y_true, y_pred)
    return matrix[0][0]/(matrix[0][0] + matrix[1][0])

In [479]:
recall(y, y_pred)

0.8

In [480]:
def f1(y_true, y_pred):
    return 2*(precision(y_true, y_pred)*recall(y_true, y_pred))/(precision(y_true, y_pred) + recall(y_true, y_pred))

In [481]:
f1(y, y_pred)

0.7272727272727272

Проверим правильность написанных функций.

In [448]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [489]:
print(accuracy(y,y_pred) == accuracy_score(y,y_pred))
print(precision(y, y_pred) == precision_score(y, y_pred))
print(recall(y,y_pred) == recall_score(y, y_pred))
print(f1(y, y_pred) == f1_score(y, y_pred))
print(conf_matrix(y,y_pred) == confusion_matrix(y, y_pred))

True
True
True
True
[[False  True]
 [ True False]]


In [491]:
conf_matrix(y,y_pred), confusion_matrix(y, y_pred)

(array([[4, 2],
        [1, 3]]),
 array([[3, 2],
        [1, 4]], dtype=int64))

Разница в матрицах вызвана различным представлением TP и TN. Я делал на основании материала, представленного в уроке. Sklearn имеет другое представление (выдержка из официальной документации):
###### Thus in binary classification, the count of true negatives is $C_{0,0}$ , false negatives is $C_{1,0}$, true positives is $C_{1,1}$ and false positives is $C_{0,1}$.


6. Могла ли модель переобучиться? Почему?

Я считаю что да, так как при очень большом количестве итераций, log loss стремится к 0 (как было видно в задании 2) и веса подгоняются под идеальные для данного набора данных и соответственно на другом наборе данных модель не сможет сделать корректного предсказания.